In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/OneDrive-JohnsHopkins/000Projects/0000-Infrastructure/0000-RecFld/RecFldGrn


# Load Data 

## Load Raw

In [2]:
# raw Data

import os 
import pandas as pd

# read the raw data here.
RecName = 'P'
folder = 'data/Simulations/'
raw_file_path =  os.path.join(folder, RecName) + '.csv'
print(raw_file_path)

# load all records
df = pd.read_csv(raw_file_path)
df

data/Simulations/P.csv


,PID,age,basicInfo
0,P0,52,Male&B
1,P1,21,Male&B
2,P2,23,Female&B
3,P3,77,Female&B
4,P4,23,Male&A
5,P5,73,Female&D
6,P6,34,Female&A
7,P7,76,Male&C


## Select Columns

In [3]:
focal_ids = ['PID']
columns = ['age', 'basicInfo']

df = df[focal_ids + columns]
df

,PID,age,basicInfo
0,P0,52,Male&B
1,P1,21,Male&B
2,P2,23,Female&B
3,P3,77,Female&B
4,P4,23,Male&A
5,P5,73,Female&D
6,P6,34,Female&A
7,P7,76,Male&C


## Basic Filtering

Cohort () & Time range ()

In [4]:
# nothing

## DF Rec Raw


with the name of `df_RecRaw`.

In [5]:
df_RecRaw = df

# Fields in Records

In [6]:
df = df_RecRaw.copy()

## Field: ID

In [7]:
RecName = 'P'
RID = RecName + 'ID'
print(RID)
df.head()

PID


,PID,age,basicInfo
0,P0,52,Male&B
1,P1,21,Male&B
2,P2,23,Female&B
3,P3,77,Female&B
4,P4,23,Male&A


## Field: age

In [8]:
df['age']

0    52
1    21
2    23
3    77
4    23
5    73
6    34
7    76
Name: age, dtype: int64

## Field: basicInfo

In [9]:
df['basicInfo']

0      Male&B
1      Male&B
2    Female&B
3    Female&B
4      Male&A
5    Female&D
6    Female&A
7      Male&C
Name: basicInfo, dtype: object

## Pin Down

In [10]:
focal_ids = [RID]
fields_cols = ['age', 'basicInfo']

df_proc = df[focal_ids + fields_cols]
df_proc

,PID,age,basicInfo
0,P0,52,Male&B
1,P1,21,Male&B
2,P2,23,Female&B
3,P3,77,Female&B
4,P4,23,Male&A
5,P5,73,Female&D
6,P6,34,Female&A
7,P7,76,Male&C


# Save and Load

## Save

In [11]:
from recfldgrn.datapoint import convert_PID_to_PIDgroup
from recfldgrn.datapoint import RANGE_SIZE, write_df_to_folders, load_df_data_from_folder
RANGE_SIZE = 10000; IDNAME = 'PID'

In [12]:
df_proc

,PID,age,basicInfo
0,P0,52,Male&B
1,P1,21,Male&B
2,P2,23,Female&B
3,P3,77,Female&B
4,P4,23,Male&A
5,P5,73,Female&D
6,P6,34,Female&A
7,P7,76,Male&C


In [13]:
# the folder to save the Record.
###############
RecName = RecName
df = df_proc
###############

rec_folder = 'Data/ProcData/RecFolder'
data_folder = os.path.join(rec_folder, RecName)
print(data_folder, '<----- fullrec_folder to save the records')

if not os.path.exists(data_folder): 
    os.makedirs(data_folder)

write_df_to_folders(RecName, data_folder, df, IDNAME)

Data/ProcData/RecFolder/P <----- fullrec_folder to save the records


## Load

In [15]:
from recfldgrn.datapoint import load_df_data_with_RecName
df = load_df_data_with_RecName(rec_folder, RecName)
print(df.shape)
df.head()

(8, 3)


,PID,age,basicInfo
0,P0,52,Male&B
1,P1,21,Male&B
2,P2,23,Female&B
3,P3,77,Female&B
4,P4,23,Male&A


## Load One Sample

In [16]:
from recfldgrn.datapoint import DataPoint
####################
PIDValue = 'P5'  
######################
Pat = DataPoint(IDNAME, PIDValue, rec_folder, RANGE_SIZE)
Pat

<PID [P5]>

In [17]:
RecName

'P'

In [18]:
Pat.get_df_rec(RecName)

,PID,age,basicInfo
0,P5,73,Female&D
